<a href="https://colab.research.google.com/github/davidfague/Model_Reduction_Methods/blob/main/NMDASimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation for generating data for dendritic spike analysis
##Recorded Currents:

Na,K,Ca,ih,...

## synapse distribution:

10,000 random. will update to include more realistic algorithm

#### Download modules from Github

In [1]:
!pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 55.6 MB/s eta 0:00:00


In [2]:
!pip install neuron_reduce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!git clone https://github.com/davidfague/Model_Reduction_Methods.git

Cloning into 'Model_Reduction_Methods'...
remote: Enumerating objects: 467, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 467 (delta 87), reused 145 (delta 85), pack-reused 314
Receiving objects: 100% (467/467), 3.89 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (238/238), done.


In [4]:
%cd Model_Reduction_Methods/

#import reduction and expansion functions
from test_neuron_reduce.subtree_reductor_func import subtree_reductor
from cable_expander_func import cable_expander

#import recording functions
from stylized_module.recorder import Recorder

#import analysis functions
from utils import make_seg_df,generate_stylized_geometry,make_reduced_seg_df,plot_morphology,check_connectivity,generate_reduced_cell_seg_coords, create_seg_var_report

import pandas as pd

/content/Model_Reduction_Methods


In [5]:
%cd expand_example

/content/Model_Reduction_Methods/expand_example


In [6]:
# compile the mod files
!nrnivmodl mod

/content/Model_Reduction_Methods/expand_example
Mod files: "mod/mod/CaDynamics_E2.mod" "mod/mod/Ca_HVA.mod" "mod/mod/Ca_LVAst.mod" "mod/mod/epsp.mod" "mod/mod/Ih.mod" "mod/mod/Im.mod" "mod/mod/K_Pst.mod" "mod/mod/K_Tst.mod" "mod/mod/Nap_Et2.mod" "mod/mod/NaTa_t.mod" "mod/mod/NaTs2_t.mod" "mod/mod/SK_E2.mod" "mod/mod/SKv3_1.mod"

Creating 'x86_64' directory for .o files.

 -> Compiling mod_func.cpp
 -> NMODL ../mod/CaDynamics_E2.mod
 -> NMODL ../mod/Ca_HVA.mod
Translating CaDynamics_E2.mod into /content/Model_Reduction_Methods/expand_example/x86_64/CaDynamics_E2.c
 -> NMODL ../mod/Ca_LVAst.mod
Thread Safe
Translating Ca_HVA.mod into /content/Model_Reduction_Methods/expand_example/x86_64/Ca_HVA.c
 -> NMODL ../mod/epsp.mod
Translating Ca_LVAst.mod into /content/Model_Reduction_Methods/expand_example/x86_64/Ca_LVAst.c
Thread Safe
Thread Safe
 -> NMODL ../mod/Ih.mod
 -> NMODL ../mod/Im.mod
Translating epsp.mod into /content/Model_Reduction_Methods/expand_example/x86_64/epsp.c
Thread Safe
Tr

## Setup smiulation parameters

In [7]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import ipywidgets as widgets
from neuron import h
from scipy import signal
from IPython.display import display, clear_output
from ipywidgets import interactive_output, HBox, VBox, Label, Layout

from __future__ import division
from neuron import gui,h
import numpy as np
import time
import matplotlib.pyplot as plt

%matplotlib inline


h.load_file('stdrun.hoc')
# h.nrn_load_dll(paths.COMPILED_LIBRARY_REDUCED_ORDER)  # choose the set of mechanisms
h.nrn_load_dll('./x86_64/.libs/libnrnmech.so')


1.0

### Create a cell with reduced morphology

In [8]:
%ls

cell1.asc  example_expand.py  L5PCtemplate.hoc  x86_64/
Cell.hoc   L5PCbiophys3.hoc   mod/


In [9]:
from abc import ABC, abstractmethod
class PointCurrent(ABC):
    """A module for current point process"""

    def __init__(self, cell, sec_index, loc=0.5):
        """
        cell: target cell object
        sec_index: index of the target section in the section list
        loc: location on a section, between [0,1]
        """
        self.cell = cell
        self.sec_index = sec_index
        self.loc = loc
        self.pp_obj = None  # point process object
        self.rec_vec = None  # vector for recording

    @abstractmethod
    def setup(self, record: bool = None) -> None:
        pass

    def setup_recorder(self):
        size = [round(h.tstop / h.dt) + 1] if hasattr(h, 'tstop') else []
        self.rec_vec = h.Vector(*size).record(self.pp_obj._ref_i)

    def get_section(self) -> h.Section:
        return self.cell.all[self.sec_index]

    def get_segment(self):
        return self.pp_obj.get_segment()

    def get_segment_id(self) -> int:
        """Get the index of the injection target segment in the segment list"""
        iseg = int(self.get_segment().x * self.get_section().nseg)
        return self.cell.sec_id_in_seg[self.sec_index] + iseg


In [10]:
from __future__ import annotations
from typing import List, Optional, Any, TYPE_CHECKING
class CurrentInjection(PointCurrent):
    """A module for current injection"""

    def __init__(self, cell, sec_index: int, loc: float = 0.5,
                 pulse: bool = True, current: Optional[np.ndarray, List[int]] = None,
                 dt: Optional[np.ndarray] = None, record: bool = False, **pulse_param: Any) -> None:
        """
        cell: target cell object
        sec_index: index of the target section in the section list
        loc: location on a section, between [0,1]
        pulse: If True, use pulse injection with keyword arguments in 'pulse_param'
               If False, use waveform resources in vector 'current' as injection
        Dt: current vector time step size
        record: If True, enable recording current injection history
        """
        super().__init__(cell, sec_index, loc)
        self.pp_obj = h.IClamp(self.get_section()(self.loc))
        self.inj_vec = None
        if pulse:
            self.setup_pulse(**pulse_param)
        else:
            if current is None:
                current = [0]
            self.setup_current(current, dt)
        self.setup(record)

    def setup(self, record: bool = False) -> None:
        if record:
            self.setup_recorder()

    def setup_pulse(self, **pulse_param: Any) -> None:
        """Set IClamp attributes. Argument keyword: attribute name, arugment value: attribute value"""
        for param, value in pulse_param.items():
            setattr(self.pp_obj, param, value)

    def setup_current(self, current: Optional[np.ndarray, List[int]], dt: Optional[np.ndarray]) -> None:
        """Set current injection with the waveform in vector 'current'"""
        ccl = self.pp_obj
        ccl.dur = 0
        ccl.dur = h.tstop if hasattr(h, 'tstop') else 1e30
        if dt is None:
            dt = h.dt
        self.inj_vec = h.Vector()
        self.inj_vec.from_python(current)
        self.inj_vec.append(0)
        self.inj_vec.play(ccl._ref_amp, dt)

In [11]:
class Synapse(PointCurrent):
    '''
    class for adding synapses
    '''
    def __init__(self, cell, stim: h.NetStim, sec_index: int,
                  syn_type: str = 'Exp2Syn', gmax: float = 0.01, loc: float = 0.5,
                  record: bool = False): #Optional[synapse]=None possible implementation for adding existing synapses
        super().__init__(cell, sec_index, loc)
        self.stim = stim
        self.gmax = gmax
        self.__synapse_type(syn_type)
        self.setup(record)

    # PRIVATE METHODS
    def __synapse_type(self, syn_type):
        if syn_type == 'AlphaSynapse1':
            # Reversal potential (mV); Synapse time constant (ms)
            self.syn_params = {'e': 0., 'tau': 2.0}
            # Variable name of maximum conductance (uS)
            self.gmax_var = 'gmax'
        elif syn_type == 'Exp2Syn':
            self.syn_params = {'e': 0., 'tau1': 1.0, 'tau2': 3.0}
            self.gmax_var = '_nc_weight'
        else:
            raise ValueError("Synpase type not defined.")
        self.syn_type = syn_type
        self.pp_obj = getattr(h, syn_type)(self.get_section()(self.loc))

    def __setup_synapse(self):
        self.syn = self.pp_obj
        self.nc = h.NetCon(self.stim, self.syn, 1, 0, 1)
        for key, value in self.syn_params.items():
            setattr(self.syn, key, value)
        self.set_gmax()

    # PUBLIC METHODS
    def setup(self, record: bool = False):
        self.__setup_synapse()
        if record:
            self.setup_recorder()
    
    def set_gmax(self, gmax: float = None):
        if gmax is not None:
            self.gmax = gmax
        if self.gmax_var == '_nc_weight':
            self.nc.weight[0] = self.gmax
        else:
            setattr(self.syn, self.gmax_var, self.gmax)

In [12]:
class Listed_Synapse():
    '''
    class for going from synapses_list and netcons_list to recording synapses
    '''
    def __init__(self,synapse_obj,netcon_objs,record=True): 
        self.pp_obj = None  # point process object
        self.rec_vec = None  # vector for recording
        self.setup(record)

        

    def setup(self,record):
        self.setup_synapse()
        if record:
            self.setup_recorder()
    
    def setup_synapse(self,synapse_obj,netcon_objs):
      self.pp_obj=synapse_obj
      self.ncs=netcon_objs

    def setup_recorder(self):
          size = [round(h.tstop/h.dt)+1] if hasattr(h,'tstop') else []
          try:
              self.rec_vec = h.Vector(*size).record(self.pp_obj._ref_igaba)
          except:
                  self.rec_vec = MultiSynCurrent()
                  vec_inmda = h.Vector(*size).record(self.pp_obj._ref_inmda)
                  vec_iampa = h.Vector(*size).record(self.pp_obj._ref_iampa)
                  self.rec_vec.add_vec(vec_inmda)
                  self.rec_vec.add_vec(vec_iampa)
                  

class MultiSynCurrent(object):
    def __init__(self):
        self.vec_list = []
    
    def add_vec(self,vec):
        self.vec_list.append(vec)
    
    def as_numpy(self):
        return np.sum(np.array([vec.as_numpy() for vec in self.vec_list]), axis=0)

In [13]:
from typing import Optional, Union, List
import math
class cell_model():
  '''expanded cell model class for ECP calculation
  takes cell model and does bookkeeping for analysis functions
  '''
  def __init__(self,model,synapses_list=None,netcons_list=None,gen_3d=True,spike_threshold: Optional[float] = None):
    self.all=model.all
    self.soma=model.soma
    self.apic=model.apic
    self.dend=model.dend
    self.axon=model.axon
    #convert nrn section lists to python lists if applicable
    self.all=self.convert_sectionlist(sectionlist=self.all)
    self.soma=self.convert_sectionlist(sectionlist=self.soma, return_singles=True) #if sectionlist contains only one section returns just the section instead of list of sections
    self.dend=self.convert_sectionlist(sectionlist=self.dend)
    self.apic=self.convert_sectionlist(sectionlist=self.apic)
    self.axon=self.convert_sectionlist(sectionlist=self.axon)
    self.spike_threshold = spike_threshold
    self.synapses_list=synapses_list #list of synapse objects from model reduction
    self.netcons_list=netcons_list # list of netcon objects from model reduction
    self.segments=[] # list for nrn segment objects
    self.injection=[] # list of injection objects
    self.synapse=[] # list of python synapse class objects
    self.__store_synapses_list() #store and record synapses from the synapses_list used to initialize the cell
    self.sec_id_lookup = {}  # dictionary from section type id to section index
    self.sec_id_in_seg = []  # index of the first segment of each section in the segment list
    self.sec_angs = [] # list of angles that were used to branch the cell
    if gen_3d==True:
      self.generate_sec_coords()
    self.__store_segments()
    self.__set_spike_recorder()
    if self.apic[2].n3d()==2: #calc_seg_coords only works for cell with sections that only have two 3d coordinates. #detailed cell has too many 3d coordinates
      self.__calc_seg_coords()
    self.grp_ids = []
    self.generate_geometry_file()

  def __calc_seg_coords(self):
    """Calculate segment coordinates for ECP calculation"""
    p0 = np.empty((self._nseg, 3))
    p1 = np.empty((self._nseg, 3))
    p05 = np.empty((self._nseg, 3))
    r = np.empty(self._nseg)
    for isec, sec in enumerate(self.all):
        iseg = self.sec_id_in_seg[isec]
        nseg = sec.nseg
        pt0 = np.array([sec.x3d(0), sec.y3d(0), sec.z3d(0)])
        pt1 = np.array([sec.x3d(1), sec.y3d(1), sec.z3d(1)])
        pts = np.linspace(pt0, pt1, 2 * nseg + 1)
        p0[iseg:iseg + nseg, :] = pts[:-2:2, :]
        p1[iseg:iseg + nseg, :] = pts[2::2, :]
        p05[iseg:iseg + nseg, :] = pts[1:-1:2, :]
        r[iseg:iseg + nseg] = sec.diam / 2
    self.seg_coords = {'dl': p1 - p0, 'pc': p05, 'r': r}

  def __store_segments(self):
    self.segments = []
    self.sec_id_in_seg = []
    nseg = 0
    for sec in self.all:
        self.sec_id_in_seg.append(nseg)
        nseg += sec.nseg
        for seg in sec:
            self.segments.append(seg)
            self.__store_point_processes(seg)
    self._nseg = nseg

  def __store_point_processes(self,seg):
    for pp in seg.point_processes():
        self.injection.append(pp)

  def __set_spike_recorder(self, threshold: Optional = None):
      if threshold is not None:
          self.spike_threshold = threshold
      if self.spike_threshold is None:
          self.spikes = None
      else:
          vec = h.Vector()
          nc = h.NetCon(self.soma(0.5)._ref_v, None, sec=self.soma)
          nc.threshold = self.spike_threshold
          nc.record(vec)
          self.spikes = vec

  def add_injection(self, sec_index, **kwargs):
        """Add current injection to a section by its index"""
        self.injection.append(CurrentInjection(self, sec_index, **kwargs))

  def add_synapse(self, stim: h.NetStim, sec_index: int, **kwargs):
        """Add synapse to a section by its index"""
        self.synapse.append(Synapse(self, stim, sec_index, **kwargs))

  def generate_sec_coords(self):
      '''
      Note: need to improve branching so that it is random direction in a quadrant of a sphere rather than x-y plane
      takes a cell that has no n3d() coordinates and gives new coordinates
      by choosing an arbitrary direction for the subtree to move
      '''
      section_obj_list= self.all
      # print(section_obj_list)
      axial=False
      parent_sections=[] #list for already seen parent_sections of this type
      for sec in section_obj_list:
        sec_length=sec.L
        if sec is self.soma:
          self.sec_angs.append(0)
          # pt0 = [0., -1 * sec.diam, 0.] #does not seem to preserve soma shape , but need to make sure children begin at correct 3d coordinate.
          # pt1 = [0., 0., 0.]
          # sec.pt3dclear()
          # sec.pt3dadd(*pt0, sec.diam)
          # sec.pt3dadd(*pt1, sec.diam)
          print('Changing soma nseg from',sec.nseg,'to 1')
          sec.nseg = 1
        else:
          if sec.parentseg() is not None:
            psec=sec.parentseg().sec
            if psec in self.apic and not self.apic[0]:
              nbranch = len(psec.children())
            else:
              nbranch=1
          else:
            psec=None # may need to provide more implementation in the case of no 3d coords and no parent section.
            nbranch=1
          rot = 2 * math.pi/nbranch
          if nbranch==1:
            i=1
          else:
            i=parent_sections.count(psec)
          parent_sections.append(psec)
          length=sec.L
          diameter=sec.diam
          fullsecname = sec.name()
          # print(fullsecname)
          sec_type = fullsecname.split(".")[1][:4]
          # print(sec_type)
          if sec_type == "apic":
            if sec==self.apic[0]:
              ang=1.570796327
            else:
              ang=np.random.uniform(low=0,high=np.pi)
          elif sec_type=="dend":
            ang=-np.random.uniform(low=0,high=np.pi)
          elif sec_type=="axon":
            ang=-1.570796327
          else:
            print(sec,sec_type,' is not apic, dend or axon')
            ang=0
          if axial == True:
            x = 0
            y = length*((ang>=0)*2-1)
          else:
            x = length * math.cos(ang)
            y = length * math.sin(ang)
          self.sec_angs.append(ang)
          #find starting position #need to update to use parent segment coordinates instead of using first section coordinate
          pt0 = [psec.x3d(1), psec.y3d(1), psec.z3d(1)]
          pt1 = [0., 0., 0.]
          pt1[1] = pt0[1] + y
          pt1[0] = pt0[0] + x * math.cos(i * rot)
          pt1[2] = pt0[2] + x * math.sin(i * rot)
          sec.pt3dclear()
          sec.pt3dadd(*pt0, sec.diam)
          sec.pt3dadd(*pt1, sec.diam)
        if int(sec.L) != int(sec_length):
          print('Error: generation of 3D coordinates resulted in change of section length for',sec,'from',sec_length,'to',sec.L)

  def generate_geometry_file(self):
    '''
    generates geometry file specifying name, pid, ang, radius, length, type
    work in progress
    '''
    df = pd.DataFrame()
    ids=[]
    names=[]
    types=[]
    pids=[]
    axials=[]
    nbranchs=[]
    Ls=[]
    Rs=[]
    # angs=self.angs
    for sec in self.all:
      # print(dir(sec))
      name=sec.name()
      # print(name)
      names.append(name)
      ids.append(names.index(name))
      _,sec_type_withinteger=name.split('.')
      sec_type,_=sec_type_withinteger.split('[')
      types.append(sec_type)
      pseg = sec.parentseg()
      if pseg == None:
        pids.append(None)
      else:
        psec=pseg.sec
        px3d=psec.x3d
        pids.append(names.index(psec.name()))
      # axials.append('TRUE')
      # nbranchs.append(1)
      Ls.append(sec.L)
      # print(dir(sec))
      Rs.append(sec.diam/2)
    df['id']=ids
    df['name']=names
    df['pid']=pids
    df['type']=types
    df['L']=Ls
    df['R']=Rs
    # df['axials']=axials # may need to fix
    # df['nbranch']=nbranchs # may need to fix
    self.geometry=df

  def convert_sectionlist(self,sectionlist,return_singles=False):
    '''
    convert nrn sectionlist objects to python list
    return_singles set to true will return section instead of [section] for lists with only one section
    '''
    new_sectionlist=[]
    if str(type(sectionlist)) == "<class 'hoc.HocObject'>":
      for sec in sectionlist:
        new_sectionlist.append(sec)
    else:
      new_sectionlist=sectionlist
    if return_singles==True:
      if str(type(new_sectionlist))!="<class 'nrn.Section'>":
        if len(new_sectionlist)==1:
          new_sectionlist=new_sectionlist[0]
    return new_sectionlist
  def __store_synapses_list(self):
    '''
    store and record synapses from the list from model reduction algorithm
    '''
    temp_list=[] # generate temp list that has each netcon's synapse obj
    for netcon in self.netcons_list:
      syn=netcon.syn()
      if syn in self.synapses_list:
        syn_seg_id=self.segments.index(netcon.syn().get_segment())
        if syn in self.segments[syn_seg_id].point_processes():
          temp_list.append(syn)
        else:
          temp_list.append(None)
          print("Warning: synapse not in designated segment's point processes")

      else:
        temp_list.append(None)
        print("Warning: potentially deleted synapse:","|NetCon obj:",netcon,"|Synapse obj:",syn,"the NetCon's synapse is not in synapses_list. Check corresponding original cell's NetCon for location, etc.")
    # now use temp list to assign each synapse its netcons
    for synapse in synapses_list:
      synapse_netcons=[]
      if synapse in temp_list:
        netcon_id=temp_list.index(synapse) #get all the netcon indices that are pointed toward this synapse # use np.where() instead of index() to return multiple indices.
        synapse_netcons.append(self.netcons[netcon_id])
        self.synapse.append(Listed_Synapse(synapse,synapse_netcons)) #record synapse and add to the list
      else:
        print('Warning: Synapse not in temp_list.')


In [14]:

# # Load biophysical parameters from a json file
# filepath = os.path.join(paths.RESOURCES_ROOT, 'biophys_parameters/ReducedOrderL5_passive.json')
# with open(filepath) as f:
#     biophys_param = json.load(f)
# for genome in biophys_param['genome']:
#     if genome['value'] != "": genome['value'] = float(genome['value'])

h.load_file('L5PCbiophys3.hoc')

# # Load morphology parameters from a csv file
# geometry = pd.read_csv(paths.GEO_REDUCED_ORDER, index_col='id')
h.load_file("import3d.hoc")


# Create a cell object
# cell = ReducedOrderL5CellPassiveDendrite(geometry=geometry, full_biophys=biophys_param)
h.load_file('L5PCtemplate.hoc')
complex_cell = h.L5PCtemplate('cell1.asc')
h.celsius = 37
h.v_init = complex_cell.soma[0].e_pas

#Add synapses to the complex model
synapses_list, netstims_list, netcons_list, randoms_list = [], [], [] ,[]

all_segments = [i for j in map(list,list(complex_cell.apical)) for i in j] + [i for j in map(list,list(complex_cell.basal)) for i in j]
len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in all_segments])
rnd = np.random.RandomState(10)
for i in range(10000):
    seg_for_synapse = rnd.choice(all_segments,   p=len_per_segment/sum(len_per_segment)) #choose a random segment with probability based on the length of segment
    synapses_list.append(h.Exp2Syn(seg_for_synapse))
    if rnd.uniform()<0.85: # 85% synapses are excitatory
        e_syn, tau1, tau2, spike_interval, syn_weight = 0, 0.3, 1.8,  1000/2.5, 0.0016
    else: #inhibitory case
        e_syn, tau1, tau2, spike_interval, syn_weight = -86, 1,   8,   1000/15.0, 0.0008
    #set synaptic varibales
    synapses_list[i].e, synapses_list[i].tau1, synapses_list[i].tau2 = e_syn, tau1, tau2
    #set netstim variables
    netstims_list.append(h.NetStim())
    netstims_list[i].interval, netstims_list[i].number, netstims_list[i].start, netstims_list[i].noise = spike_interval, 9e9, 100, 1
    #set random
    randoms_list.append(h.Random())
    randoms_list[i].Random123(i)
    randoms_list[i].negexp(1)
    netstims_list[i].noiseFromRandom(randoms_list[i])       
    #set netcon varibales 
    netcons_list.append(h.NetCon(netstims_list[i], synapses_list[i] ))
    netcons_list[i].delay, netcons_list[i].weight[0] = 0, syn_weight
print(synapses_list)
#reduce dendritic trees to cables
reduced_cell, synapses_list, netcons_list, txt = subtree_reductor(complex_cell, synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)
# print('reduced cell n3d')
# for sec in reduced_cell.dend:
#   print(sec,sec.n3d())

#expand cables to idealized dendritic trees
sections_to_expand = [reduced_cell.hoc_model.apic[0]]
furcations_x=[0.289004]
nbranches=[4]
reduced_dendritic_cell, synapses_list, netcons_list, txt = cable_expander(reduced_cell, sections_to_expand, furcations_x, nbranches, 
                                                                          synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)

# # try branching the branches
# sections_to_expand = [reduced_dendritic_cell.apic[1],reduced_dendritic_cell.apic[2],reduced_dendritic_cell.apic[3],reduced_dendritic_cell.apic[4]]
# furcations_x=[0.50,0.50,0.50,0.50]
# nbranches=[4,4,4,4]
# reduced_dendritic_cell, synapses_list, netcons_list, txt = cable_expander(reduced_dendritic_cell, sections_to_expand, furcations_x, nbranches, 
#                                                                           synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)


# print('dendritic cell n3d')
# for sec in reduced_dendritic_cell.all:
#   print(sec, sec.n3d())

#use defined cell_model class for generating 3d coordinates, recording ECP, 'book-keeping' etc...
cell = cell_model(reduced_dendritic_cell,synapses_list=synapses_list,netcons_list=netcons_list,spike_threshold = 10)
# cell._nbranch=4

[Exp2Syn[0], Exp2Syn[1], Exp2Syn[2], Exp2Syn[3], Exp2Syn[4], Exp2Syn[5], Exp2Syn[6], Exp2Syn[7], Exp2Syn[8], Exp2Syn[9], Exp2Syn[10], Exp2Syn[11], Exp2Syn[12], Exp2Syn[13], Exp2Syn[14], Exp2Syn[15], Exp2Syn[16], Exp2Syn[17], Exp2Syn[18], Exp2Syn[19], Exp2Syn[20], Exp2Syn[21], Exp2Syn[22], Exp2Syn[23], Exp2Syn[24], Exp2Syn[25], Exp2Syn[26], Exp2Syn[27], Exp2Syn[28], Exp2Syn[29], Exp2Syn[30], Exp2Syn[31], Exp2Syn[32], Exp2Syn[33], Exp2Syn[34], Exp2Syn[35], Exp2Syn[36], Exp2Syn[37], Exp2Syn[38], Exp2Syn[39], Exp2Syn[40], Exp2Syn[41], Exp2Syn[42], Exp2Syn[43], Exp2Syn[44], Exp2Syn[45], Exp2Syn[46], Exp2Syn[47], Exp2Syn[48], Exp2Syn[49], Exp2Syn[50], Exp2Syn[51], Exp2Syn[52], Exp2Syn[53], Exp2Syn[54], Exp2Syn[55], Exp2Syn[56], Exp2Syn[57], Exp2Syn[58], Exp2Syn[59], Exp2Syn[60], Exp2Syn[61], Exp2Syn[62], Exp2Syn[63], Exp2Syn[64], Exp2Syn[65], Exp2Syn[66], Exp2Syn[67], Exp2Syn[68], Exp2Syn[69], Exp2Syn[70], Exp2Syn[71], Exp2Syn[72], Exp2Syn[73], Exp2Syn[74], Exp2Syn[75], Exp2Syn[76], Exp2Syn[

ValueError: ignored

In [ ]:
# for sec in cell.all:
#   for seg in sec:
#     for pp in seg.point_processes():
#       print(pp)


In [ ]:
# dir(cell.dend[0](0.5).point_processes()[0]) #no way to go from segment to all synapses
# #have to go compare netcons to synapse list

In [ ]:
# dir(cell.dend[1](0.5))

In [ ]:
# #potentially deleted synapses?
# for netcon in netcons_list:
#   if netcon.syn() not in synapses_list:
#     print(netcon)
#     print(netcon.syn() in synapses_list)
#     print(netcon.syn())

In [ ]:
# for syn in synapses_list:
#   print(syn,syn.get_segment().point_processes())

In [ ]:
# #create second original model

# original_cell = h.L5PCtemplate('cell1.asc')
# synapses_list, netstims_list, netcons_list, randoms_list = [], [], [] ,[]

# all_segments = [i for j in map(list,list(original_cell.apical)) for i in j] + [i for j in map(list,list(original_cell.basal)) for i in j]
# len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in all_segments])
# rnd = np.random.RandomState(10)
# for i in range(10000):
#     seg_for_synapse = rnd.choice(all_segments,   p=len_per_segment/sum(len_per_segment)) #choose a random segment with probability based on the length of segment
#     synapses_list.append(h.Exp2Syn(seg_for_synapse))
#     if rnd.uniform()<0.85: # 85% synapses are excitatory
#         e_syn, tau1, tau2, spike_interval, syn_weight = 0, 0.3, 1.8,  1000/2.5, 0.0016
#     else: #inhibitory case
#         e_syn, tau1, tau2, spike_interval, syn_weight = -86, 1,   8,   1000/15.0, 0.0008
#     #set synaptic varibales
#     synapses_list[i].e, synapses_list[i].tau1, synapses_list[i].tau2 = e_syn, tau1, tau2
#     #set netstim variables
#     netstims_list.append(h.NetStim())
#     netstims_list[i].interval, netstims_list[i].number, netstims_list[i].start, netstims_list[i].noise = spike_interval, 9e9, 100, 1
#     #set random
#     randoms_list.append(h.Random())
#     randoms_list[i].Random123(i)
#     randoms_list[i].negexp(1)
#     netstims_list[i].noiseFromRandom(randoms_list[i])       
#     #set netcon varibales 
#     netcons_list.append(h.NetCon(netstims_list[i], synapses_list[i] ))
#     netcons_list[i].delay, netcons_list[i].weight[0] = 0, syn_weight
# original_model = cell_model(original_cell,gen_3d=False,spike_threshold = 10)
# print(synapses_list)

In [ ]:
make_reduced_seg_df(cell,"segments_expanded.csv") #need to improve make_reduced_seg_df
expanded_segments_df=pd.read_csv("segments_expanded.csv")
plot_morphology(expanded_segments_df,"expanded_morphology.svg")

# change to complex cell
# make_reduced_seg_df(cell,"segments_expanded.csv") #need to improve make_reduced_seg_df
# expanded_segments_df=pd.read_csv("segments_expanded.csv")
# plot_morphology(expanded_segments_df,"expanded_morphology.svg")

In [ ]:
# dir(cell)

In [ ]:
import os

RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !pip install neuron==8.0.0 &> /dev/null
    os.chdir('/content')
    if not os.path.isdir('Stylized-Single-Cell-and-Extracellular-Potential'):
        !git clone https://github.com/chenziao/Stylized-Single-Cell-and-Extracellular-Potential.git &> /dev/null 
    os.chdir('Stylized-Single-Cell-and-Extracellular-Potential')
    %ls

In [ ]:
from typing import Union, List, Tuple


class Recorder(object):
    """A module for recording variables"""

    def __init__(self, obj_list: Union[object, List[object], Tuple[object], np.ndarray], var_name: str = 'v') -> None:
        """
        obj_list: list of (or a single) target objects
        var_name: string of variable to be recorded
        """
        self.single = not isinstance(obj_list, (list, tuple, np.ndarray))
        self.obj_list = obj_list
        self.var_name = var_name
        self.vectors = None
        self.setup_recorder()

    def setup_recorder(self) -> None:
        size = [round(h.tstop / h.dt) + 1] if hasattr(h, 'tstop') else []
        attr_name = '_ref_' + self.var_name
        if self.single:
            self.vectors = h.Vector(*size).record(getattr(self.obj_list, attr_name))
        else:
            self.vectors = [h.Vector(*size).record(getattr(obj, attr_name)) for obj in self.obj_list]

    def as_numpy(self, copy: bool = True) -> np.ndarray:
        """
        Return a numpy 2d-array of recording, n objects-by-time
        Return a 1d-array if a single object is being recorded
        """
        if self.single:
            x = self.vectors.as_numpy()
        else:
            x = np.array([v.as_numpy() for v in self.vectors])
        if copy:
            x = x.copy()
        return x

In [ ]:
#insert unused channels for recorder

In [ ]:
for sec in cell.all:
  if not hasattr(sec(0.5),'gNaTa_t_NaTa_t'):
    sec.insert('NaTa_t')
    for seg in sec:
      seg.NaTa_t.gNaTa_tbar=0
    print(sec)

In [ ]:
for sec in cell.all:
  if not hasattr(sec(0.5),'ica_Ca_LVAst'):
    sec.insert('Ca_LVAst')
    for seg in sec:
      seg.Ca_LVAst.gCa_LVAstbar=0
    print(sec)

In [ ]:
for sec in cell.all:
  if not hasattr(sec(0.5),'ica_Ca_HVA'):
    sec.insert('Ca_HVA')
    for seg in sec:
      seg.Ca_HVA.gCa_HVAbar

In [ ]:
for sec in cell.all:
  if not hasattr(sec(0.5),'ihcn_Ih'):
    sec.insert('Ih')
    for seg in sec:
      seg.Ih.gIhbar

In [ ]:
# tstop = 6000  # ms simulation duration
tstop = 5000
dt = 0.1  # ms. does not allow change


h.tstop = tstop
h.dt = dt
h.steps_per_ms = 1/h.dt

nseg = len(cell.segments)
numTstep = int(tstop/dt)


# gNaTa_T = Recorder(obj_list = original_model.segments, var_name = 'gNaTa_t_NaTa_t')
# ical = Recorder(obj_list = original_model.segments, var_name = 'ica_Ca_LVAst')
# icah = Recorder(obj_list = original_model.segments, var_name = 'ica_Ca_HVA')
# ih = Recorder(obj_list = original_model.segments, var_name = 'ihcn_Ih')
# Vm = Recorder(obj_list = original_model.segments)

gNaTa_T = Recorder(obj_list = cell.segments, var_name = 'gNaTa_t_NaTa_t')
ical = Recorder(obj_list = cell.segments, var_name = 'ica_Ca_LVAst')
icah = Recorder(obj_list = cell.segments, var_name = 'ica_Ca_HVA')
ih = Recorder(obj_list = cell.segments, var_name = 'ihcn_Ih')
Vm = Recorder(obj_list = cell.segments)

In [ ]:
timestart=time.time()
h.run()
timestop=time.time()
t = h.t # was t=h.t() but 'float' object not callable
elapsedtime=timestop-timestart
simtime=tstop/1000 #convert from ms to s
# totaltime= elapsedtime+elapseddeftime
print('It took',round(elapsedtime),'sec to run a',simtime,'sec simulation.')
# print('The total runtime was',round(totaltime),'sec')

In [ ]:
ih_data = ih.as_numpy()
gNaTa_T_data = gNaTa_T.as_numpy()
icah_data = icah.as_numpy()
ical_data = ical.as_numpy()
Vm = Vm.as_numpy()

In [ ]:
import h5py
def createsegtracereport(reportname,dataname):
  try:
    os.remove(reportname) # reportname was string " "
  except:
    x = 1

  f = h5py.File(reportname,'w') #create a file in the w (write) mode #reportname was string ' '
  v = f.create_dataset("report/biophysical/data", data = dataname)
  f.close()

In [ ]:
# for syn in synapses_list:
#   print(syn.e)

In [ ]:
# dir(cell)

In [ ]:
# cell.segments

In [ ]:
#since synapse list has combined synapses for computational efficiency, we must use NetCon list/unique spike trains to determine how many synapses were mapped
NetCon_per_seg=[0]*nseg
inh_NetCon_per_seg=[0]*nseg
exc_NetCon_per_seg=[0]*nseg

i_NMDA_bySeg= [[0] * (numTstep+1) ] * nseg # need to implement inmda recording

v_rest=-60 #choose v_rest for categorizing inh/exc synapses

for netcon in netcons_list:
  syn=netcon.syn()
  if syn in synapses_list:
    syn_seg_id=cell.segments.index(netcon.syn().get_segment())
    if syn in cell.segments[syn_seg_id].point_processes():
      NetCon_per_seg[syn_seg_id]+=1 # get synapses per segment
      # NetCon_per_seg[syn_seg_id].append(netcon) # possible implementation if needing objects per segment
      if syn.e > v_rest:
        exc_NetCon_per_seg[syn_seg_id]+=1
        # exc_NetCon_per_seg[syn_seg_id].append(netcon)# possible implementation if needing objects per segment
      else:
        inh_NetCon_per_seg[syn_seg_id]+=1
        # inh_NetCon_per_seg[syn_seg_id].append(netcon)# possible implementation if needing objects per segment
    else:
      print("Warning: synapse not in designated segment's point processes")

  else:
    print("Warning: potentially deleted synapse:","|NetCon obj:",netcon,"|Synapse obj:",syn,"the NetCon's synapse is not in synapses_list. Check corresponding original cell's NetCon for location, etc.")

#extract inmda from each segment
for synapse in cell.synapse:
  try:
    i_NMDA = np.array(synapse.rec_vec.vec_list[1])            #current = numpy array of NEURON Vector of current NMDA current at synapse j  
    seg = synapse.get_segment_id()                            #seg = the segment in which synapse j is located 
    #print('first try')
    
    try:
      i_NMDA_bySeg[seg] = i_NMDA_bySeg[seg] + i_NMDA    
      #print('second try')                                           #Sum current over each segment
    except: 
      pass                                                                   #Except needed as some synpases do not have NMDA currrent and throw an error when called
  except:
    pass

In [ ]:

numSyn = len(cell.synapse)

excSynPerSeg = [0]*nseg
inhSynPerSeg = [0]*nseg
excSynPerSegL = [0]*nseg
inhSynPerSegL = [0]*nseg
SynParentSeg = []
SourcePop = []
SynType = []
SynDist = []

i_NMDA_bySeg= [[0] * (numTstep+1) ] * nseg

#print(len(sim.cells[0].injection))
for j in range(numSyn):
  seg = cell.synapse[j].get_segment_id()
  SynParentSeg.append(seg)
  # SynType.append(AllSegType[seg])
  # SynDist.append(AllSegDist[seg])

  if(cell.synapse[j].syntype == 'exc'):
    excSynPerSeg[seg] += 1
    SourcePop.append('exc_stim')
  else:
    inhSynPerSeg[seg] += 1
    SourcePop.append('dist_inh_stim')

  try:
    i_NMDA = np.array(cell.injection[j].rec_vec.vec_list[1])            #current = numpy array of NEURON Vector of current NMDA current at synapse j  
    seg = cell.synapse[j].get_segment_id()                            #seg = the segment in which synapse j is located 
    #print('first try')
    
    try:
      i_NMDA_bySeg[seg] = i_NMDA_bySeg[seg] + i_NMDA    
      #print('second try')                                           #Sum current over each segment
    except: 
      x = j                                                                     #Except needed as some synpases do not have NMDA currrent and throw an error when called
  except:
    x = j

In [ ]:
i_NMDA_bySeg[0]

In [ ]:
#Should add "syn per seg" to seg data frame #also can try doing SynPerSeg/segL to have more of a density metric
def plotSynDensity(SynPerSeg):
  plt.figure(figsize=(4,10))
  ax = plt.scatter(expanded_segments_df["Coord X"], expanded_segments_df["Coord Y"],c = SynPerSeg[0:nseg],cmap='jet',)
  plt.vlines(110,400,500)
  plt.text(0,450,'100 um')
  plt.hlines(400,110,210)
  plt.text(110,350,'100 um')
  plt.xticks([])
  plt.yticks([])
  cbar = plt.colorbar()
  cbar.ax.set_ylabel('Synapses per segment', rotation=270)

  plt.box(False)
  plt.savefig('exc_syns.svg')

def plotSynDensityNoSoma(SynPerSeg):
  plt.figure(figsize=(4,10))
  ax = plt.scatter(expanded_segments_df["Coord X"][1:], expanded_segments_df["Coord Y"][1:],c = SynPerSeg[1:nseg],cmap='jet',)
  plt.vlines(110,400,500)
  plt.text(0,450,'100 um')
  plt.hlines(400,110,210)
  plt.text(110,350,'100 um')
  plt.xticks([])
  plt.yticks([])
  cbar = plt.colorbar()
  cbar.ax.set_ylabel('Synapses per segment', rotation=270)

  plt.box(False)
  plt.savefig('inh_syns.svg')

In [ ]:
plotSynDensity(exc_NetCon_per_seg) #len(cell.segments) #low number of segments makes plot look odd.

In [ ]:
plotSynDensityNoSoma(inh_NetCon_per_seg) # if soma has too many inh synapses the heatmap is not as interpretable

In [ ]:
i_NMDA_df = pd.DataFrame(i_NMDA_bySeg) * 1000

In [ ]:
output_folder_name="output_control"+str(int(h.tstop))#+modelname  #include model name in output foler name (ex. original cell, reduced cell, expanded cell)

In [ ]:
#create output folder

In [ ]:
# %cd output_control

In [ ]:
createsegtracereport('v_report.h5', Vm.T)
createsegtracereport('Ca_HVA.ica_report.h5',icah_data.T)
createsegtracereport('Ca_LVAst.ica_report.h5',ical_data.T)
createsegtracereport('Ih.ihcn_report.h5',ih_data.T)
createsegtracereport('inmda_report.h5',i_NMDA_df.T)
createsegtracereport('NaTa_t.gNaTa_t_report.h5',gNaTa_T_data.T)

In [ ]:
# %cd ..

In [ ]:
#adjust so that a folder is generated for complex and reduced cell and with simulation time as part of the folder name
#maybe include syn distribution in name too